In [1]:
import pandas as pd
import math as ma

In [2]:
data=pd.read_csv("film2.csv")

In [3]:
data

,critique,Lady,Snakes,Luck,Superman,Dupree,Night
0,Lisa Rose,2.5,3.5,3.0,3.5,2.5,3.0
1,Gene Seymour,3.0,3.5,1.5,5.0,3.5,3.0
2,Michael Phillips,2.5,3.0,NaN,3.5,NaN,4.0
3,Claudia Puig,NaN,3.5,3.0,4.0,2.5,4.5
4,Mick Lasalle,3.0,4.0,2.0,3.0,2.0,3.0
5,Jack Matthews,3.0,4.0,NaN,5.0,3.5,3.0
6,Toby,NaN,4.5,NaN,4.0,1.0,NaN
7,Anne,1.5,NaN,4.0,NaN,2.0,NaN


In [4]:
data.fillna("No rating", inplace = True)

In [5]:
data[data['critique']=='Toby']

,critique,Lady,Snakes,Luck,Superman,Dupree,Night
6,Toby,No rating,4.5,No rating,4,1,No rating


### Manhattan

In [6]:
def sim_distanceManhattan(pers1, pers2):
    pers1 = data[data['critique']==pers1]
    pers1= pers1.reset_index(drop=True)
    pers2 = data[data['critique']==pers2]
    pers2= pers2.reset_index(drop=True)
    diff=0
    for i in range(1, pers1.shape[1]):
        if (pers1.iloc[0,i]=="No rating" or pers2.iloc[0,i] =="No rating"):
            continue
        else:
            diff+=abs(pers1.iloc[0,i]-pers2.iloc[0,i])
    return diff

In [7]:
sim_distanceManhattan('Lisa Rose', 'Toby')

3.0

In [8]:
sim_distanceManhattan('Lisa Rose', 'Gene Seymour')

4.5

### Euclidean

In [9]:
def sim_distanceEuclidienne(pers1, pers2):
    pers1 = data[data['critique']==pers1]
    pers1= pers1.reset_index(drop=True)
    pers2 = data[data['critique']==pers2]
    pers2= pers2.reset_index(drop=True)
    summ=0
    for i in range(1, pers1.shape[1]):
        if (pers1.iloc[0,i]=="No rating" or pers2.iloc[0,i] =="No rating"):
            continue
        else:
            summ+= ((pers1.iloc[0,i]-pers2.iloc[0,i])**2)
    return ma.sqrt(summ)

In [10]:
sim_distanceEuclidienne('Lisa Rose', 'Gene Seymour')

2.3979157616563596

In [11]:
sim_distanceEuclidienne('Lisa Rose', 'Toby')

1.8708286933869707

### Nearest Neighbor

In [12]:
Critiques = [ ]
for i in range(len(data)):
    Critiques.append(data.iloc[i,0])

In [13]:
Critiques

['Lisa Rose',
 'Gene Seymour',
 'Michael Phillips',
 'Claudia Puig',
 'Mick Lasalle',
 'Jack Matthews',
 'Toby',
 'Anne']

In [14]:
def computeNearestNeighbor(nouveauCritique, Critiques): 
    distances=[ ]
    for i in Critiques:
        if nouveauCritique != i:
            distances.append((sim_distanceManhattan(i, nouveauCritique),i))
            distances.sort()
        else :
            continue
    return distances

In [15]:
computeNearestNeighbor('Lisa Rose', Critiques)

[(1.5, 'Michael Phillips'),
 (2.0, 'Claudia Puig'),
 (2.5, 'Anne'),
 (3.0, 'Mick Lasalle'),
 (3.0, 'Toby'),
 (3.5, 'Jack Matthews'),
 (4.5, 'Gene Seymour')]

In [16]:
def computeNearestNeighbor_Euclidean(nouveauCritique, Critiques): 
    distances=[ ]
    for i in Critiques:
        if nouveauCritique != i:
            distances.append((sim_distanceEuclidienne(i, nouveauCritique),i))
            distances.sort()
        else :
            continue
    return distances

In [17]:
computeNearestNeighbor_Euclidean('Lisa Rose', Critiques)

[(1.118033988749895, 'Michael Phillips'),
 (1.4142135623730951, 'Mick Lasalle'),
 (1.5, 'Anne'),
 (1.5811388300841898, 'Claudia Puig'),
 (1.8708286933869707, 'Toby'),
 (1.9364916731037085, 'Jack Matthews'),
 (2.3979157616563596, 'Gene Seymour')]

### Raccomandation

In [18]:
def recommend(nouveauCritique):
    nouveauCritique = data[data['critique']==nouveauCritique ]
    nouveauCritique= nouveauCritique.reset_index(drop=True)
    nn= [ ]
    nn= computeNearestNeighbor(nouveauCritique.iloc[0,0], Critiques)
    best=nn[0][1]
    critique = data[data['critique']==nn[0][1]]
    critique= critique.reset_index(drop=True)
    if all(nouveauCritique.iloc[0] !="No rating" )==True:
        print('Il/elle a vu tous les films')
        return
    else:
        for i in range(1, nouveauCritique.shape[1]):
            if (nouveauCritique.iloc[0,i]=="No rating" and critique.iloc[0,i] !="No rating"):
                print('Film recommandé: ' +str(data.columns[i])+', Vote: '+str(critique.iloc[0,i])) 
        return

In [19]:
def recommend_mod(nouveauCritique):
    nouveauCritique = data[data['critique']==nouveauCritique ]
    nouveauCritique= nouveauCritique.reset_index(drop=True)
    nn= [ ]
    nn= computeNearestNeighbor(nouveauCritique.iloc[0,0], Critiques)
    recc=[]
    critique = data[data['critique']==nn[0][1]]
    critique= critique.reset_index(drop=True)
    if all(nouveauCritique.iloc[0] !="No rating" )==True:
        print('Il/elle a vu tous les films')
        return
    else:
        for i in range(1, nouveauCritique.shape[1]):
            if (nouveauCritique.iloc[0,i]=="No rating" and critique.iloc[0,i] !="No rating"):
                recc.append((critique.iloc[0,i], data.columns[i]))
                #recc.sort()
                #print('Film recommandé: ' +str(data.columns[i])+', Vote: '+str(critique.iloc[0,i])) 
        return max(recc)

In [20]:
recommend_mod('Toby')

(4.0, 'Luck')

In [21]:
recommend('Lisa Rose')

Il/elle a vu tous les films


In [22]:
recommend('Anne')

Film recommandé: Snakes, Vote: 3.0
Film recommandé: Superman, Vote: 3.5
Film recommandé: Night, Vote: 4.0


In [23]:
recommend('Toby')

Film recommandé: Lady, Vote: 1.5
Film recommandé: Luck, Vote: 4.0


### Raccomandation b ii

In [24]:
def recommend_complet(nouveauCritique, film):
    nouveauCritique = data[data['critique']==nouveauCritique ]
    nouveauCritique= nouveauCritique.reset_index(drop=True)
    total=0
    sim=0
    if (nouveauCritique.iloc[0][film]=="No rating" ):
        for j in Critiques:
            critique= data[data['critique']==j]
            critique= critique.reset_index(drop=True)
            if (j!=nouveauCritique.iloc[0,0] and critique.iloc[0][film] !="No rating"):
                    total+=(1/(1+sim_distanceManhattan(nouveauCritique.iloc[0,0], j)))*critique.iloc[0][film]
                    sim+=(1/(1+sim_distanceManhattan(nouveauCritique.iloc[0,0], j)))
        print('Total:', total)
        print('S(a):', sim)
        print('Total/s(a):', total/sim)
    else :
        print('Film déjà voté')
    return  

In [25]:
recommend_complet('Anne', 'Night')

Total: 6.535347985347985
S(a): 1.811782661782662
Total/s(a): 3.6071368399770862


In [26]:
def recommend_complet_mod(nouveauCritique, film):
    nouveauCritique = data[data['critique']==nouveauCritique ]
    nouveauCritique= nouveauCritique.reset_index(drop=True)
    total=0
    sim=0
    if (nouveauCritique.iloc[0][film]=="No rating" ):
        for j in Critiques:
            critique= data[data['critique']==j]
            critique= critique.reset_index(drop=True)
            if (j!=nouveauCritique.iloc[0][film] and critique.iloc[0][film] !="No rating"):
                    total+=(1/(1+sim_distanceManhattan(nouveauCritique.iloc[0,0], j)))*critique.iloc[0][film]
                    sim+=(1/(1+sim_distanceManhattan(nouveauCritique.iloc[0,0], j)))
    return  (total/sim)

In [27]:
def Bestrecommend(nouveauCritique):
    nouveauCritique = data[data['critique']==nouveauCritique ]
    nouveauCritique= nouveauCritique.reset_index(drop=True)
    res=[ ]
    for i in range(1, nouveauCritique.shape[1]):
        if (nouveauCritique.iloc[0,i]=="No rating" ):
            res.append((recommend_complet_mod(nouveauCritique.iloc[0,0], nouveauCritique.columns[i]),nouveauCritique.columns[i]))
    return max(res)

In [28]:
Bestrecommend('Anne')

(3.908627564898196, 'Superman')

### Pearson

In [29]:
def pearson(pers1, pers2):
    pers1 = data[data['critique']==pers1]
    pers1= pers1.reset_index(drop=True)
    pers2 = data[data['critique']==pers2]
    pers2= pers2.reset_index(drop=True)
    sum_xy=0
    sum_x=0 
    sum_y=0
    sum_x2=0
    sum_y2=0
    n=0
    for i in range(1, pers1.shape[1]):
        if (pers1.iloc[0,i] !="No rating" and pers2.iloc[0,i] !="No rating"):
            n += 1 
            x=pers1.iloc[0,i] 
            y=pers2.iloc[0,i] 
            sum_xy +=x*y 
            sum_x += x 
            sum_y += y
            sum_x2 += x**2 
            sum_y2 += y**2
    denominator = ma.sqrt(sum_x2 - (sum_x**2) / n) * ma.sqrt(sum_y2 - (sum_y**2) / n)
    if denominator == 0: 
        return 0
    else:
        return (sum_xy - (sum_x * sum_y) /n ) / denominator

In [30]:
pearson('Lisa Rose', 'Gene Seymour')

0.39605901719066977

In [31]:
def PearsonRecommend(nouveauCritique):
    nouveauCritique = data[data['critique']==nouveauCritique ]
    nouveauCritique= nouveauCritique.reset_index(drop=True)
    res=[ ]
    best=0
    best_film=[ ]
    for j in Critiques:
        if(j!=nouveauCritique.iloc[0,0]):
            critique= data[data['critique']==j]
            critique= critique.reset_index(drop=True)
            res.append((pearson(nouveauCritique.iloc[0,0], j),j))
    best= (max(res)[1])
    prox=data[data['critique']==best]
    prox=prox.reset_index(drop=True)
    for i in range(1, nouveauCritique.shape[1]):
        if (nouveauCritique.iloc[0,i]=="No rating" and prox.iloc[0,i] !='No rating'):
            best_film.append((prox.iloc[0,i], nouveauCritique.columns[i]))
    return (max(best_film))

In [32]:
PearsonRecommend('Anne')

(5.0, 'Superman')

### Cosinus

In [33]:
def cosinus(pers1, pers2):
    pers1 = data[data['critique']==pers1]
    pers1= pers1.reset_index(drop=True)
    pers2 = data[data['critique']==pers2]
    pers2= pers2.reset_index(drop=True)
    sum_xy=0
    sum_x=0 
    sum_y=0
    cos=0
    for i in range(1, pers1.shape[1]):
        if (pers1.iloc[0,i] !="No rating" and pers2.iloc[0,i] !="No rating"):
            x=pers1.iloc[0,i] 
            y=pers2.iloc[0,i] 
            sum_xy +=x*y 
            sum_x += x 
            sum_y += y
            cos= sum_xy/ (ma.sqrt(sum_x**2)* ma.sqrt(sum_y**2))
    return cos

In [34]:
cosinus('Anne', 'Lisa Rose')

0.3458333333333333

In [35]:
def CosinusRecommend(nouveauCritique):
    nouveauCritique = data[data['critique']==nouveauCritique ]
    nouveauCritique= nouveauCritique.reset_index(drop=True)
    res=[ ]
    best=0
    best_film=[ ]
    for j in Critiques:
        if(j!=nouveauCritique.iloc[0,0]):
            critique= data[data['critique']==j]
            critique= critique.reset_index(drop=True)
            res.append((cosinus(nouveauCritique.iloc[0,0], j),j))
    best= (max(res)[1])
    prox=data[data['critique']==best]
    prox=prox.reset_index(drop=True)
    for i in range(1, nouveauCritique.shape[1]):
        if (nouveauCritique.iloc[0,i]=="No rating" and prox.iloc[0,i] !='No rating'):
            best_film.append((prox.iloc[0,i], nouveauCritique.columns[i]))
    return (max(best_film))

In [36]:
CosinusRecommend('Anne')

(4.5, 'Snakes')

### Raccomandation 5

In [37]:
def SuperRecommend(nouveauCritique):
    r1=0
    r2=0
    r3=0
    nouveauCritique = data[data['critique']==nouveauCritique ]
    nouveauCritique= nouveauCritique.reset_index(drop=True)
    for i in range(1, nouveauCritique.shape[1]):
        if (nouveauCritique.iloc[0,i]=="No rating"):
            r1= PearsonRecommend(nouveauCritique.iloc[0,0])
            r2= CosinusRecommend(nouveauCritique.iloc[0,0])
            r3=recommend_mod(nouveauCritique.iloc[0,0])
    if(r1==0 and r2==0 and r3==0):
        print('Il/elle a vu tous les films')
        return
    print('Pearson: ',r1)
    print('Cosinus: ',r2)
    print('Manhattan: ',r3)
    return  

In [38]:
 SuperRecommend('Anne')

Pearson:  (5.0, 'Superman')
Cosinus:  (4.5, 'Snakes')
Manhattan:  (4.0, 'Night')


In [39]:
 SuperRecommend('Lisa Rose')

Il/elle a vu tous les films


### New dataset

In [47]:
data=pd.read_csv("new_film_mod.csv")

In [48]:
data.fillna("No rating", inplace = True)

In [49]:
data

,critique,Lady,Snakes,Luck,Superman,Dupree,Night
0,Pippo,1,0.5,3.5,No rating,2.5,1.0
1,Gennaro,2,No rating,1,3,5,0.5
2,Genevieve,No rating,2.5,No rating,No rating,2,1.0
3,Flavienne,5,No rating,3.5,4,No rating,4.5
4,Maccio,3,5,1.5,3,1,3.5


In [50]:
Critiques = [ ]
for i in range(len(data)):
    Critiques.append(data.iloc[i,0])

In [51]:
 SuperRecommend('Maccio')

Il/elle a vu tous les films


In [52]:
SuperRecommend('Genevieve')

Pearson:  (3.0, 'Superman')
Cosinus:  (5.0, 'Lady')
Manhattan:  (3.5, 'Luck')


### Conclusion

#### Pearson

D'après la mesure de Pearson, la critique Genevieve a donné des notes similaires à celles de Gennaro. Pearson est la mesure la plus efficace mais la plus complexe en terme de calcul  

#### Cosinus

Pour cosinus, Flavienne est la critique la plus proche de Genevieve cependant il n'y a pas de notes similaires avec Genevieve. Dans cet exemple, la mesure cosinus n'est pas optimale

#### Manhattan

Nous avons des données manquantes donc la distance de Manhattan n'est pas la plus efficace 